In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graph

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-08 23:53:02--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-04-08 23:53:02 (6.57 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|          review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|         US|    1797882|R3I2DHQBR577SS|B001ANOOOE|       2102612|The Naked Bee Vit...|          Beauty|          5|            0|          0|   N|                Y|          Five Stars| Love this, excell...| 2015-08-31|
|         US|   18381298|R1QNE9NQFJC2Y4|B0016J22EQ|     106393691|Alba Botanica Sun...|          Beauty|          5|

In [5]:
# Drop null values
df = df.dropna()

# Create the vine_table dataframe.
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3I2DHQBR577SS|          5|            0|          0|   N|                Y|
|R1QNE9NQFJC2Y4|          5|            0|          0|   N|                Y|
|R3LIDG2Q4LJBAO|          5|            0|          0|   N|                Y|
|R3KSZHPAEVPEAL|          5|            0|          0|   N|                Y|
| RAI2OIG50KZ43|          5|            0|          0|   N|                Y|
|R1R30FA4RB5P54|          4|            0|          0|   N|                Y|
|R30IJKCGJBGPJH|          5|            0|          0|   N|                Y|
|R18GLJJPVQ1OVH|          5|            0|          0|   N|                Y|
| R8TVYIJXLYJT0|          5|            0|          0|   N|                Y|
|R1CJGF6M3PVHEZ|          1|            0|          2|   N|     

In [6]:
# Create a new DataFrame or table where the total_votes count is equal to or greater than 20.
total_votes = vine_df.filter("total_votes >= 20")
total_votes.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2QRUE9REK8OUC|          5|           18|         23|   N|                Y|
|R2POXRW6PXHXZG|          5|           25|         30|   N|                Y|
| RZN43XRZ89IIJ|          3|           10|         27|   N|                N|
|R29Q748WSHZ3SN|          5|           23|         24|   N|                Y|
| R46UL5G5HEPRZ|          5|           18|         20|   N|                N|
|R2UQHSNWU6WTZX|          5|           74|         76|   N|                N|
|R2HSRI3D6E2M9Y|          5|           69|         71|   N|                Y|
|R17ARFSDV555EW|          1|           29|         29|   N|                Y|
|R3GL5156FFEDQA|          5|           20|         21|   N|                Y|
|R29KQJC9PIYWQS|          5|           25|         26|   N|     

In [7]:
# Retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
helpful_votes = vine_df.filter("(helpful_votes/total_votes)>=0.5")
helpful_votes.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2A30ALEGLMCGN|          4|            1|          1|   N|                Y|
| R6CE3SOIUJGP4|          4|            1|          1|   N|                N|
|R3I4BQ6654MQNA|          5|            1|          1|   N|                Y|
|R3LUPG356F1D40|          5|            3|          3|   N|                N|
|R2QRUE9REK8OUC|          5|           18|         23|   N|                Y|
|R1BMJXQGYXQG2F|          5|            1|          1|   N|                N|
|R1V3QWZ5919M3K|          5|            4|          5|   N|                Y|
| RW70OUSBVWMLT|          5|            1|          2|   N|                N|
|R19RE4JO8O8D56|          5|            1|          1|   N|                N|
| RWBW8IV6X4XBI|          5|            1|          1|   N|     

In [8]:
# Retrieve all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
paid_vine_df = helpful_votes.filter("vine == 'Y'")
paid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R12YOP0B139RNC|          5|            2|          2|   Y|                N|
|R1HNDB3KRVOIN9|          4|            1|          1|   Y|                N|
| RHOGS1MC49YF9|          4|            1|          1|   Y|                N|
|R2VMGRFWD95FVA|          4|            2|          2|   Y|                N|
| RU8FQJU8MOIAJ|          4|            1|          1|   Y|                N|
|R2MCBVX7KD9JIH|          5|            1|          1|   Y|                N|
| RKEEXI87JBD50|          5|            2|          2|   Y|                N|
|R3O4E0AWIETFPY|          4|            1|          2|   Y|                N|
|R16NWNUWU13YEU|          3|            1|          1|   Y|                N|
|R231U7D1CIJPDV|          1|            1|          2|   Y|     

In [9]:
# Retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
unpaid_vine_df = helpful_votes.filter("vine == 'N'")
unpaid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2A30ALEGLMCGN|          4|            1|          1|   N|                Y|
| R6CE3SOIUJGP4|          4|            1|          1|   N|                N|
|R3I4BQ6654MQNA|          5|            1|          1|   N|                Y|
|R3LUPG356F1D40|          5|            3|          3|   N|                N|
|R2QRUE9REK8OUC|          5|           18|         23|   N|                Y|
|R1BMJXQGYXQG2F|          5|            1|          1|   N|                N|
|R1V3QWZ5919M3K|          5|            4|          5|   N|                Y|
| RW70OUSBVWMLT|          5|            1|          2|   N|                N|
|R19RE4JO8O8D56|          5|            1|          1|   N|                N|
| RWBW8IV6X4XBI|          5|            1|          1|   N|     

In [10]:
# The total number of reviews (paid).
total_paid_reviews = paid_vine_df.count()
total_paid_reviews

11597

In [11]:
# The number of 5-star reviews (paid).
five_star_paid = paid_vine_df.filter(paid_vine_df.star_rating == 5).count()
five_star_paid

4168

In [12]:
# The percentage of 5-star reviews (paid).
paid_review_percentage = (five_star_paid / total_paid_reviews) * 100
paid_review_percentage

35.940329395533325

In [13]:
# The total number of reviews (unpaid).
total_unpaid_reviews = unpaid_vine_df.count()
total_unpaid_reviews

1726832

In [14]:
# The number of 5-star reviews (unpaid).
five_star_unpaid = unpaid_vine_df.filter(unpaid_vine_df.star_rating == 5).count()
five_star_unpaid

1047970

In [15]:
# The percentage of 5-star reviews (unpaid).
unpaid_review_percentage = (five_star_unpaid / total_unpaid_reviews) * 100
unpaid_review_percentage

60.6874322458699